# Luyện mô hình

In [38]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  7 16:15:55 2022

@author: SkyMap
"""

# from email.mime import base
# from osgeo import gdal
import numpy as np
import pandas as pd
# import geopandas as gp
import os, glob

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, ReLU
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from keras.models import load_model
    
import rasterio
from sklearn import datasets
np.random.seed()

Using TensorFlow backend.


lấy thông tin của label và nodata, chuyển hết dữ liệu về dạng flatten

In [39]:
def get_index_and_mask_train(fp_mask, nodata_value=100):
    src = rasterio.open(fp_mask)
    mask = src.read()[0].flatten()
    index_nodata = np.where(mask == nodata_value)
    mask_train = np.delete(mask, index_nodata)
    return mask_train, index_nodata

chuyển dữ liệu về dạng dataframe, trong đó xóa luôn dữ liệu nodata để giảm kích thước DF

In [40]:
def get_df_flatten_train(fp_img, list_number_band, index_nodata):
    src = rasterio.open(fp_img)
    # return to img train
    list_band_have = list(range(1,src.count+1))
    dfObj = pd.DataFrame()
    if set(list_number_band).issubset(list_band_have):
        img = src.read(list_number_band)
        i = 0
        for band in img:
            band = band.flatten()
            band = np.delete(band, index_nodata)
            name_band = f"band {list_number_band[i]}"
            dfObj[name_band] = band
            i+=1
        return dfObj
    else:
        miss = np.setdiff1d(list_number_band, list_band_have)
        print("*"*15, "ERROR", "*"*15)
        print(f"Image dont have band : {miss.tolist()}")

In [41]:
def create_data_train_from_ones_img(fp_img, fp_mask, list_band_to_train, size_get=3000):
    mask_train, index_nodata = get_index_and_mask_train(fp_mask)
    df_dataset = get_df_flatten_train(fp_img, list_band_to_train, index_nodata)
    df_dataset['label'] = mask_train


    g = df_dataset.groupby('label', group_keys=False)
    g = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))
    g = pd.DataFrame(g)
    # size_get = 3000       # sample size
    replace = False  # with replacement
    fn = lambda obj: obj.loc[np.random.choice(obj.index, size_get, replace),:]
    a= g.groupby('label', as_index=False).apply(fn)
    # print(a)
    return pd.DataFrame(a)

In [42]:
def create_data_train_all_img(list_fp_img, list_fp_mask, list_band_to_train, out_fp_csv_train):
    print(list_fp_img)
    dir_name_img = os.path.dirname(list_fp_img[0])
    list_df_all = []
    for fp_mask in list_fp_mask:
        base_name = os.path.basename(fp_mask)
        fp_img = os.path.join(dir_name_img, base_name)
        df_tmp = create_data_train_from_ones_img(fp_img, fp_mask, list_band_to_train)
        # print(df_tmp)
        list_df_all.append(df_tmp)
        result = pd.concat(list_df_all)
        # print(result)
    # print(np.unique(result['label'].to_numpy()))
    result.to_csv(out_fp_csv_train)

In [43]:
def create_data_train(csv_training):
    datasets = pd.read_csv(csv_training).iloc[:, 2:]
    print(datasets.shape)
    X = datasets.iloc[:, :-1]
    Y = datasets.iloc[:, -1]
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    Y = np_utils.to_categorical(encoded_Y)
    return X,Y

In [44]:
# out_fp_csv_train = r'E:\WORK\Cambodia_HatDieu\Data\training_fix_012.csv'
# datasets = pd.read_csv(out_fp_csv_train).iloc[:, 2:]
# X = datasets.iloc[:, :-1]
# Y = datasets.iloc[:, -1]
# encoder = LabelEncoder()
# encoder.fit(Y)
# encoded_Y = encoder.transform(Y)
# Y = np_utils.to_categorical(encoded_Y)
# Y

# Mạng DENSE

In [45]:
def train(input, label, classes=7, epochs=100, batch_size=100, shuffle=True, model_path='model.h5'):
    assert classes>=2, 'number classese must be more than 1'
    model = Sequential()
    model.add(Dense(8, input_dim = 4))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Dense(10, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Dense(10, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Dense(10, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(ReLU())

    # model.add(Dense(10, activation = 'relu'))
    # model.add(BatchNormalization())
    # model.add(ReLU())

    # model.add(Dense(10, activation = 'relu'))
    # model.add(BatchNormalization())
    # model.add(ReLU())
    if classes==2:
        model.add(Dense(2, activation = 'sigmoid'))
        loss = 'binary_crossentropy'
    else:
        model.add(Dense(classes, activation = 'softmax'))
        loss='categorical_crossentropy'

    model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
    model.fit(input, label, epochs=epochs, batch_size=batch_size, shuffle=shuffle)
    scores = model.evaluate(input, label)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    model.save(model_path)

In [46]:
def main():
    list_fp_img = glob.glob(r'E:\WORK\Cambodia_HatDieu\Data\img\*.tif')
    list_fp_mask = glob.glob(r'E:\WORK\Cambodia_HatDieu\Data\mask_fix_012\*.tif')
    list_band_to_train = [1,2,3,4]
    out_fp_csv_train = r'E:\WORK\Cambodia_HatDieu\Data\training_fix_012_3000.csv'
    model_path=r"E:\WORK\Cambodia_HatDieu\Data\model_3000.h5"

    if not os.path.exists(out_fp_csv_train):
        create_data_train_all_img(list_fp_img, list_fp_mask, list_band_to_train, out_fp_csv_train)
    X, Y = create_data_train(out_fp_csv_train)
    print(X.shape)
    print(Y.shape)
    print('Traing...')
    train(X, Y, classes=3, epochs=1000, batch_size=100000, shuffle=True, model_path=model_path)



Training mạng Dense

In [47]:
main()

(75000, 5)
(75000, 4)
(75000, 3)
Traing...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 1/1000
75000/75000 [==============================] - 2s 32us/step - loss: 1.0960 - accuracy: 0.5669
Epoch 2/1000
75000/75000 [==============================] - 0s 1us/step - loss: 1.0759 - accuracy: 0.5762
Epoch 3/1000
75000/75000 [==============================] - 0s 1us/step - loss: 1.0600 - accuracy: 0.6782
Epoch 4/1000
75000/75000 [==============================] - 0s 1us/step - loss: 1.0446 - accuracy: 0.6832
Epoch 5/1000
75000/75000 [==============================] - 0s 1us/step - loss: 1.0299 - accuracy: 0.6877


# Mạng XGBOOST

Create data training with 80% train and 20%test

In [ ]:
import numpy as np
import pandas as pd

def create_data_train_Xgboost(csv_training, training_per = 0.8):
    datasets = pd.read_csv(csv_training).iloc[:, 2:]
    ds_train = datasets.sample(frac=training_per)
    ds_test = datasets[~datasets.isin(ds_train)].dropna()

    X_train = ds_train.iloc[:, :-1]
    Y_train = ds_train.iloc[:, -1]
    X_test = ds_test.iloc[:, :-1]
    Y_test = ds_test.iloc[:, -1]

    return X_train, Y_train, X_test, Y_test

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
import pickle

def XGboost_train():
    list_fp_img = glob.glob(r'E:\WORK\Cambodia_HatDieu\Data\img\*.tif')
    list_fp_mask = glob.glob(r'E:\WORK\Cambodia_HatDieu\Data\mask_fix_012\*.tif')
    list_band_to_train = [1,2,3,4]
    out_fp_csv_train = r'E:\WORK\Cambodia_HatDieu\Data\training_fix_012_3000.csv'
    fp_model_save = r"E:\WORK\Cambodia_HatDieu\Data\model_3000_v2.pkl"
    if not os.path.exists(out_fp_csv_train):
        create_data_train_all_img(list_fp_img, list_fp_mask, list_band_to_train, out_fp_csv_train)
    X_train, Y_train, X_test, Y_test = create_data_train_Xgboost(out_fp_csv_train, training_per = 0.8)
    print('Training ...')
    clf = GradientBoostingClassifier(n_estimators=10000, learning_rate=1.0, max_depth=3, random_state=0).fit(X=X_train, y=Y_train)
    print('acc: ',clf.score(X_test, Y_test))
    with open(fp_model_save, 'wb') as file:
        pickle.dump(clf, file)

In [ ]:
XGboost_train()

acc:  0.8974


In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(r"E:\WORK\Cambodia_HatDieu\agaricus.txt.test")
dtest = xgb.DMatrix(r"E:\WORK\Cambodia_HatDieu\agaricus.txt.train.txt")
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

[13:07:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
import pandas as pd
import numpy as np
data = pd.DataFrame(np.arange(12).reshape((4,3)), columns=['a', 'b', 'c'])
label = pd.DataFrame(np.random.randint(2, size=4))
data


,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11


In [ ]:
dtrain = xgb.DMatrix(data, label=label)

## XGBOOST WITH LIB FAST

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import glob, os



In [ ]:
def create_data_train_Xgboost(csv_training, training_per = 0.8):
    datasets = pd.read_csv(csv_training).iloc[:, 2:]
    ds_train = datasets.sample(frac=training_per)
    ds_test = datasets[~datasets.isin(ds_train)].dropna()

    X_train = ds_train.iloc[:, :-1]
    Y_train = ds_train.iloc[:, -1]
    X_test = ds_test.iloc[:, :-1]
    Y_test = ds_test.iloc[:, -1]

    return X_train, Y_train, X_test, Y_test

In [ ]:

list_fp_img = glob.glob(r'E:\WORK\Cambodia_HatDieu\Data\img\*.tif')
list_fp_mask = glob.glob(r'E:\WORK\Cambodia_HatDieu\Data\mask_fix_012\*.tif')
list_band_to_train = [1,2,3,4]
out_fp_csv_train = r'E:\WORK\Cambodia_HatDieu\Data\training_fix_012_9000.csv'
fp_model_save = r"E:\WORK\Cambodia_HatDieu\Data\model_9000_v2.model"
if not os.path.exists(out_fp_csv_train):
    create_data_train_all_img(list_fp_img, list_fp_mask, list_band_to_train, out_fp_csv_train)
X_train, Y_train, X_test, Y_test = create_data_train_Xgboost(out_fp_csv_train, training_per = 0.8)
print('Training ...')
dtrain = xgb.DMatrix(X_train, label=Y_train)
dtest = xgb.DMatrix(X_test, label=Y_test)
num_round = 10
param = {'max_depth': 2, 'eta': 1, 'objective': 'multi:softmax'}
param['nthread'] = 4
param['eval_metric'] = 'auc'
param['num_class'] = 3
evallist = [(dtest, 'eval'), (dtrain, 'train')]
bst = xgb.train(param, dtrain, num_round, evallist)
bst.save_model(fp_model_save)


Training ...
[0]	eval-auc:0.92782	train-auc:0.92583
[1]	eval-auc:0.94900	train-auc:0.94712
[2]	eval-auc:0.95662	train-auc:0.95468
[3]	eval-auc:0.95988	train-auc:0.95865
[4]	eval-auc:0.96275	train-auc:0.96164
[5]	eval-auc:0.96355	train-auc:0.96246
[6]	eval-auc:0.96464	train-auc:0.96350
[7]	eval-auc:0.96490	train-auc:0.96392
[8]	eval-auc:0.96593	train-auc:0.96532
[9]	eval-auc:0.96777	train-auc:0.96745
